# Ceneo Scraper

## Imports

In [15]:
import requests
from bs4 import  BeautifulSoup

# Send request to Ceneo.pl service

In [16]:
product_id = "108290707"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews_scroll"
response = requests.get(url)
response.status_code

200

## Convert plain text HTML code to DOM structure

In [17]:
page_dom = BeautifulSoup(response.text, "html.parser")
opinions = page_dom.select("div.js_product-review")
opinion = page_dom.select_one("div.js_product-review")
print(opinion)

<div class="user-post user-post__card js_product-review" data-entry-id="17349010">
<header class="user-post__header">
<div class="js_lazy user-post__avatar user-rank__avatar" data-bg="/Content/img/account/avatar/6.svg"></div>
</header>
<div class="user-post__body">
<div class="user-post__content">
<span class="user-post__author-name">
KamilZ1</span>
<span class="user-post__author-recomendation">
<em class="recommended">Polecam</em>
</span>
<span class="user-post__score">
<span class="screen-reader-text">Ocena:</span>
<span class="score-container score-container--s js_score-container">
<span class="score-marker score-marker--s" style="width: 100.00%;"></span>
</span>
<span class="user-post__score-count">5/5</span>
<span class="user-post__published">
        Wystawiono
        <time datetime="2023-03-31 23:03:01">2 lata temu, </time>
<time datetime="2021-08-05 17:01:22">po 2 latach</time> użytkowania            </span>
</span>
<div class="user-post__text">W obudowie przypominającej niewi

## Extract components for single opinion

|Component|Selector|Variable|
|---------|--------|--------|
|opinion ID|["data-entry-id"]|opinion_id|
|opinion’s author|span.user-post__author-name|author|
|author’s recommendation|span.user-post__author-recomendation em|recommendation|
|score expressed in number of stars|span.user-post__score-count|score|
|opinion’s content|div.user-post__text|content|
|list of product advantages|div.review-feature--positives ~ div.review-feature__item|advantages|
|list of product disadvantages|div.review-feature--negatives ~ div.review-feature__item |disadvantages|
|how many users think that opinion was helpful|button.vote-yes > span|helpful|
|how many users think that opinion was unhelpful|button.vote-no > span|not_helpful|
|publishing date|span.user-post__published > time:nth-child(1)["datetime"]|pub_date|
|purchase date|span.user-post__published > time:nth-child(2)["datetime"]|pur_date|



In [32]:
def extra_content(ancestor, selector, attribute = None):
    if attribute:
        try:
           return ancestor.select_one(selector)[attribute].strip()
        except TypeError:
            return None
    return ancestor.select_one(selector).text.strip()

In [34]:
opinion_id = opinion["data-entry-id"]
author = extra_content(opinion, "span.user-post__author-name")
recommendation = extra_content(opinion,"span.user-post__author-recomendation > em")
score = extra_content(opinion,"span.user-post__score-count")
content = extra_content(opinion, "div.user-post__text")
pros = [p.text.strip() for p in opinion.select("div.review-featuretitle--positives ~ div.review-featureitem")]
cons = [p.text.strip() for c in opinion.select("div.review-featuretitle--negatives ~ div.review-featureitem")]
helpful = extra_content(opinion,"button.vote-yes > span")
unhelpful = extra_content(opinion,"button.vote-no > span")
publish_date = extra_content(opinion, "span.user-post__published > time:nth-child(1)","datetime")
purchase_date = extra_content(opinion, "span.user-post__published > time:nth-child(2)","datetime")

TypeError: Tag.select_one() takes from 2 to 3 positional arguments but 4 were given